In [1]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

# Xopt class, Astra kekgun example

This is the class method for running Xopt.

In [2]:
from xopt import Xopt

In [3]:
# Make a proper input file. 
YAML="""
xopt: {output_path: temp, verbose: true,
  algorithm: random_sampler}
  
algorithm:
  name: random_sampler
  options: {verbose: T, chunk_size: 4, max_samples: 10} 

simulation:
  name: astra_with_generator
  evaluate: astra.evaluate.evaluate_astra_with_generator

# Note that you can call another file in the top level group:
vocs: ../templates/kekgun/kekgun_vocs.yaml

"""

In [4]:
!mkdir temp/

mkdir: temp/: File exists


In [5]:
# Create object
X = Xopt(config=YAML)

# Change some things to make it run faster
X.vocs['constants']['lspch'] = True
X.vocs['constants']['ipart'] = 1000
X.vocs['constants']['zstop'] = 0.2

# Show config
X

Loading config as text



            Xopt 
________________________________           
Version: 0.4.0
Configured: True
Config as YAML:
xopt: {output_path: /Users/chrisonian/Code/GitHub/xopt/examples/xopt_class/temp, verbose: true,
  algorithm: random_sampler}
algorithm:
  name: random_sampler
  function: xopt.sampler.random_sampler
  options: {verbose: T, chunk_size: 4, max_samples: 10}
simulation:
  name: astra_with_generator
  evaluate: astra.evaluate.evaluate_astra_with_generator
  options: {archive_path: null, merit_f: null, astra_input_file: /Users/chrisonian/Code/GitHub/xopt/examples/templates/kekgun/kekgun.in,
    generator_input_file: /Users/chrisonian/Code/GitHub/xopt/examples/templates/kekgun/dist004.in}
vocs:
  name: kekgun
  description: Test case
  simulation: astra_with_generator
  templates: {astra_input_file: /Users/chrisonian/Code/GitHub/xopt/examples/templates/kekgun/kekgun.in,
    generator_input_file: /Users/chrisonian/Code/GitHub/xopt/examples/templates/kekgun/dist004.in}
  variables:
   

In [6]:
# Check random inputs 
X.random_inputs()

{'sig_x': 0.6067410524828215,
 'lt': 0.037680024377719154,
 'maxe(1)': 44.338837775791596,
 'phi(1)': 0.8034849774319923,
 'maxb(1)': 0.28708203334241034,
 'maxe(2)': 26.43333845476326,
 'phi(2)': -177.18330864754023,
 'maxb(2)': 0.0857364785124242,
 'maxe(3)': 0.2720477022099068,
 'maxe(4)': 29.52503359778697,
 'phi(3)': 22.096924408955203,
 'phi(4)': -35.003186222747246,
 'phi(6)': 19.96187062326794,
 'ipart': 1000,
 'lspch': True,
 'zstop': 0.2}

In [7]:
%%time
# Do a random evaluate to check that everything will run
output = X.random_evaluate()
output

CPU times: user 70.3 ms, sys: 15.3 ms, total: 85.6 ms
Wall time: 5.86 s


{'error': False,
 'end_mean_z': 0.2,
 'end_norm_emit_x': 6.9611e-07,
 'end_core_emit_95percent_x': 4.7392e-07,
 'end_core_emit_90percent_x': 3.9755e-07,
 'end_core_emit_80percent_x': 2.8725e-07,
 'end_norm_emit_y': 6.9115e-07,
 'end_core_emit_95percent_y': 4.7490999999999996e-07,
 'end_core_emit_90percent_y': 3.9987e-07,
 'end_core_emit_80percent_y': 2.888e-07,
 'end_norm_emit_z': 2.5739,
 'end_core_emit_95percent_z': 1.9239e-06,
 'end_core_emit_905percent_z': 1.6122e-06,
 'end_core_emit_80percent_z': 1.2376e-06,
 'end_mean_t': 7.7264e-10,
 'end_mean_x': -2.7579999999999997e-07,
 'end_sigma_x': 0.00066569,
 'end_sigma_xp': 0.0025523,
 'end_cov_x__xp': 1.687990133e-06,
 'end_mean_y': -3.6354e-07,
 'end_sigma_y': 0.00066497,
 'end_sigma_yp': 0.0025488,
 'end_cov_y__yp': 1.683903531e-06,
 'end_mean_kinetic_energy': 1395200.0,
 'end_sigma_z': 0.0014167000000000001,
 'end_sigma_energy': 9207.0,
 'end_cov_z__energy': 12.7871342,
 'end_n_particle_loss': 0,
 'end_total_charge': 9.9899999999999

In [8]:
# These are the algorithm options
X.algorithm['options']

{'verbose': 'T', 'chunk_size': 4, 'max_samples': 10}

In [9]:
# These are the options in the evaluate function
X.simulation['options']

{'archive_path': None,
 'merit_f': None,
 'astra_input_file': '/Users/chrisonian/Code/GitHub/xopt/examples/templates/kekgun/kekgun.in',
 'generator_input_file': '/Users/chrisonian/Code/GitHub/xopt/examples/templates/kekgun/dist004.in'}

# Run Sampler using processes or threads

In [10]:
from concurrent.futures import ProcessPoolExecutor as PoolExecutor
#from concurrent.futures import ThreadPoolExecutor as PoolExecutor
executor = PoolExecutor()

In [11]:
# Show config
X.run(executor)

Starting at time 2020-04-23T08:59:24-07:00


███████╗ █████╗ ███╗   ███╗██████╗ ██╗     ███████╗██████╗ 
██╔════╝██╔══██╗████╗ ████║██╔══██╗██║     ██╔════╝██╔══██╗
███████╗███████║██╔████╔██║██████╔╝██║     █████╗  ██████╔╝
╚════██║██╔══██║██║╚██╔╝██║██╔═══╝ ██║     ██╔══╝  ██╔══██╗
███████║██║  ██║██║ ╚═╝ ██║██║     ███████╗███████╗██║  ██║
╚══════╝╚═╝  ╚═╝╚═╝     ╚═╝╚═╝     ╚══════╝╚══════╝╚═╝  ╚═╝
                                                           

Xopt Random Sampler
Version 0.4.0

....4 samples completed in 0.12715 minutes
Samples written to:
 /Users/chrisonian/Code/GitHub/xopt/examples/xopt_class/temp/sampler-2020-04-23-1.json
....4 samples completed in 0.10864 minutes
Samples written to:
 /Users/chrisonian/Code/GitHub/xopt/examples/xopt_class/temp/sampler-2020-04-23-2.json
...

In [12]:
# Check for errors
X.population['error']

[False, False, False, False, False, False, False, False, False, False, False]

# Write this configuration

In [13]:
X.save('test.yaml')

Config written as YAML to test.yaml


In [14]:
!cat test.yaml

xopt: {output_path: /Users/chrisonian/Code/GitHub/xopt/examples/xopt_class/temp, verbose: true,
  algorithm: random_sampler}
algorithm:
  name: random_sampler
  function: xopt.sampler.random_sampler
  options: {verbose: T, chunk_size: 4, max_samples: 10}
simulation:
  name: astra_with_generator
  evaluate: astra.evaluate.evaluate_astra_with_generator
  options: {archive_path: null, merit_f: null, astra_input_file: /Users/chrisonian/Code/GitHub/xopt/examples/templates/kekgun/kekgun.in,
    generator_input_file: /Users/chrisonian/Code/GitHub/xopt/examples/templates/kekgun/dist004.in}
vocs:
  name: kekgun
  description: Test case
  simulation: astra_with_generator
  templates: {astra_input_file: /Users/chrisonian/Code/GitHub/xopt/examples/templates/kekgun/kekgun.in,
    generator_input_file: /Users/chrisonian/Code/GitHub/xopt/examples/templates/kekgun/dist004.in}
  variables:
    sig_x: [0.05, 1]
    lt: [0.005, 0.07]
    maxe(1): [20, 50]
    phi(1): [-30, 30]
    maxb(1): [0, 0.4]
    m

# Run with MPI

In [15]:
!mpirun -n 4 python -m mpi4py.futures -m xopt.mpi.run test.yaml

  _   
                | |  
__  _____  _ __ | |_ 
\ \/ / _ \| '_ \| __|
 >  < (_) | |_) | |_ 
/_/\_\___/| .__/ \__|
          | |        
          |_|        

_________________________________
Parallel execution with 4 workers
Loading as YAML file: test.yaml

            Xopt 
________________________________           
Version: 0.4.0
Configured: True
Config as YAML:
xopt: {output_path: /Users/chrisonian/Code/GitHub/xopt/examples/xopt_class/temp, verbose: true,
  algorithm: random_sampler}
algorithm:
  name: random_sampler
  function: xopt.sampler.random_sampler
  options: {verbose: T, chunk_size: 4, max_samples: 10}
simulation:
  name: astra_with_generator
  evaluate: astra.evaluate.evaluate_astra_with_generator
  options: {archive_path: null, merit_f: null, astra_input_file: /Users/chrisonian/Code/GitHub/xopt/examples/templates/kekgun/kekgun.in,
    generator_input_file: /Users/chrisonian/Code/GitHub/xopt/examples/templates/kekgun/dist004.in}
vocs:
  name: kekgun
  description: Te

# Dask 

In [16]:
from dask.distributed import Client
executor = Client()
#executor = Client(processes=True)
executor

Client Scheduler: tcp://127.0.0.1:60238 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 17.18 GB


In [17]:
# Wait a few seconds for the Dask cluster to start
from time import sleep
sleep(5)

In [18]:
# Run some more
X.run(executor)

Starting at time 2020-04-23T09:00:42-07:00


███████╗ █████╗ ███╗   ███╗██████╗ ██╗     ███████╗██████╗ 
██╔════╝██╔══██╗████╗ ████║██╔══██╗██║     ██╔════╝██╔══██╗
███████╗███████║██╔████╔██║██████╔╝██║     █████╗  ██████╔╝
╚════██║██╔══██║██║╚██╔╝██║██╔═══╝ ██║     ██╔══╝  ██╔══██╗
███████║██║  ██║██║ ╚═╝ ██║██║     ███████╗███████╗██║  ██║
╚══════╝╚═╝  ╚═╝╚═╝     ╚═╝╚═╝     ╚══════╝╚══════╝╚═╝  ╚═╝
                                                           

Xopt Random Sampler
Version 0.4.0

....4 samples completed in 0.16706 minutes
Samples written to:
 /Users/chrisonian/Code/GitHub/xopt/examples/xopt_class/temp/sampler-2020-04-23-5.json
....4 samples completed in 0.13226 minutes
Samples written to:
 /Users/chrisonian/Code/GitHub/xopt/examples/xopt_class/temp/sampler-2020-04-23-6.json
...

In [19]:
X.population['error']

[False, False, False, False, False, False, False, False, False, False, False]

# Load into Pandas

In [20]:
import pandas as pd
from glob import glob
import json

In [21]:
jfiles = glob('temp/*json')
jfiles

['temp/sampler-2020-04-23-6.json',
 'temp/sampler-2020-04-23-1.json',
 'temp/sampler-2020-04-23-3.json',
 'temp/sampler-2020-04-23-2.json',
 'temp/sampler-2020-04-23-5.json',
 'temp/sampler-2020-04-23-4.json']

In [22]:
# Load a file
data = json.load(open(jfiles[0]))

data['inputs']
data['outputs']
data['error']

[False, False, False, False]

In [23]:
inputs = pd.DataFrame(data['inputs'])
inputs

,sig_x,lt,maxe(1),phi(1),maxb(1),maxe(2),phi(2),maxb(2),maxe(3),maxe(4),phi(3),phi(4),phi(6),ipart,lspch,zstop
0,0.547824,0.024510,33.353055,-3.937211,0.323803,6.147543,42.900027,0.287283,21.019288,0.013771,-13.808743,-23.825785,12.113985,1000,True,0.2
1,0.322796,0.050366,36.731695,0.731224,0.292484,14.493486,129.526765,0.132482,7.534472,22.802011,-1.452359,4.567261,-2.726451,1000,True,0.2
2,0.155174,0.048542,25.089854,27.283045,0.233000,4.219193,-147.466603,0.080899,17.143812,14.977390,-3.675056,-41.732276,43.115561,1000,True,0.2
3,0.757704,0.057092,23.634542,19.249859,0.306951,4.547130,-40.919216,0.088452,13.478526,5.770959,5.755176,35.046172,-5.517069,1000,True,0.2


In [24]:
outputs = pd.DataFrame(data['outputs'])
outputs

,error,end_mean_z,end_norm_emit_x,end_core_emit_95percent_x,end_core_emit_90percent_x,end_core_emit_80percent_x,end_norm_emit_y,end_core_emit_95percent_y,end_core_emit_90percent_y,end_core_emit_80percent_y,...,end_sigma_yp,end_cov_y__yp,end_mean_kinetic_energy,end_sigma_z,end_sigma_energy,end_cov_z__energy,end_n_particle_loss,end_total_charge,end_higher_order_energy_spread,fingerprint
0,False,0.2,4.566800e-07,3.070200e-07,2.586300e-07,1.865600e-07,4.470300e-07,3.062700e-07,2.587600e-07,1.867600e-07,...,0.001678,8.123700e-07,2019100.0,0.002071,4649.5,-7.437624,0,9.990000e-11,1180.906684,c571f17afd258526cb9773aab5125c1c
1,False,0.2,2.643100e-07,1.852600e-07,1.563000e-07,1.121800e-07,2.576200e-07,1.848400e-07,1.554200e-07,1.109400e-07,...,0.001766,8.215227e-07,2211400.0,0.004310,14057.0,25.304753,0,9.990000e-11,330.325405,f1878994998ea2ccdccc0c5cd63269c2
2,False,0.2,3.577100e-06,2.856800e-06,2.332700e-06,1.561600e-06,3.601400e-06,2.876500e-06,2.340500e-06,1.558800e-06,...,0.007502,1.329471e-05,1305200.0,0.007103,122370.0,866.020018,0,9.990000e-11,966.648295,dc2cf88c71d70b1152b044c0e374a2a6
3,False,0.2,4.677500e-07,3.724000e-07,3.188400e-07,2.381100e-07,4.647700e-07,3.739300e-07,3.204600e-07,2.378500e-07,...,0.001271,8.224608e-08,1351000.0,0.006164,72838.0,443.703212,0,9.990000e-11,528.905334,4ca21147a0018a6c08ed58fd4e82f021


# Cleanup

In [25]:
!rm -r dask-worker-space

In [26]:
!rm test.yaml

In [27]:
!rm NORRAN

In [28]:
!rm -r temp